In [27]:
import requests
import re
from bs4 import BeautifulSoup as bs
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import pandas as pd
from urllib.request import urlopen
from urllib.parse import quote
import time

# 위치 지정
driver = webdriver.Chrome("d:/py_data/chromedriver.exe")
keys = Keys()

driver.get("https://section.cafe.naver.com/")

keyword = input("찾으시려는 키워드를 입력하세요:")
END_PAGE = int(input("마지막 페이지를 입력하세요: ")) + 1
PAGE_url = "&p="

query_elm = driver.find_element_by_css_selector("#header > div.snb_area > div > form > fieldset > div > div > input")
query_elm.click() # 검색창 클릭
query_elm.send_keys(keyword) #검색어 입력

# 엔터키 입력
query_elm.send_keys(keys.ENTER)
driver.set_page_load_timeout(30) # 로드 대기시간
driver.implicitly_wait(10)  # 브라우저에서 사용되는 엔진 자체에서 파싱되는 시간을 기다려 주는 메소드

# 전체글 클릭
driver.find_element_by_css_selector("#app > div > div.container > div > div.SectionSearchContent > div.section_search_tab_list > a:nth-child(3)").click()
url = driver.current_url
driver.implicitly_wait(10) 

title_lst = []
doc_lst = []
comment_lst = []

for now_page in range(1,END_PAGE):
    driver.get(url+PAGE_url+str(now_page))
    time.sleep(2)     # 프로세스 자체를 지정한 시간동안 기다려 주는 역할(무조건 대기)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    item_lst = soup.find_all("div",class_="item_inner")
    
    for i in range(1,len(item_lst)+1):
        driver.find_element_by_css_selector("#app > div > div.container > div > div.SectionSearchContent > div.section_search_content > div > div.article_list_area > ul > li:nth-child("+str(i)+") > div > div > div > a").click()
        driver.implicitly_wait(10) 
        driver.switch_to.window(driver.window_handles[-1])
        driver.switch_to_frame('cafe_main')
        driver.implicitly_wait(20) 
        html = driver.page_source#소스 가져오기
        soup = bs(html, 'html.parser')
        
        
        try:     
            title = re.sub('[^a-zA-Zㄱ-ㅣ가-힣0-9 ]',"",soup.find('h3',class_='title_text').text)  
        except:
            title=" "
            
        title_lst.append(title)
        
        try:
            doc_temp = re.sub('[^a-zA-Zㄱ-ㅣ가-힣0-9 ]',"",soup.find("div",class_="se-module").text)
        except:
            doc_temp = " "
        doc_lst.append(doc_temp)
        
        comment_area = soup.find_all("div", class_="comment_area")
        
        comment = ""
        for i in comment_area:
            try:
                comment_temp = i.find("span",class_="text_comment").text
            except AttributeError:
                pass
            comment = comment + re.sub('[^a-zA-Zㄱ-ㅣ가-힣 ]', '',comment_temp)
        comment_lst.append(comment)
        
        driver.close()
        driver.switch_to.window(driver.window_handles[0]) # 맨 처음 탭으로 변경(0번 탭)
        driver.implicitly_wait(2) 

driver.close()
df = pd.DataFrame({"title":title_lst, "doc":doc_lst, "comment":comment_lst})
df.to_csv("./data/"+keyword+" 네이버 카페 크롤링.csv",encoding="cp949")

찾으시려는 키워드를 입력하세요:안마기
마지막 페이지를 입력하세요: 1


<ipython-input-27-30fcc5995831>:50: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame('cafe_main')
